## Análisis - "Las cinco grandes ligas" temporada 2015/2016 (masculina)

Tras hacer un análisis de cada una de las cinco grandes ligas durante la temporada 2015/2016, en este notebook en cuestión, vamos a analizar las cinco grandes ligas conjuntamente, es decir, suponiendo que las 5 grandes ligas son una única liga por así decirlo. Este análisis nos va a ayudar a entender que características son más importantes a la hora de tomar la decisión por parte del modelo de ML para decidir si el equipo va a ganar, perder o empatar el partido.

Cargamos los datos de los partidos para entrenar el modelo y poder hacer el estudio de explicabilidad.

In [1]:
import os
import pandas as pd

output_dir = "data/reduced/"
filename = "Top_5_leagues.csv"
output_path = os.path.join(output_dir, filename)

if os.path.exists(output_path):
    print(f"Matches from the competition loaded from {output_path}")
    matches_in_Top5Leagues = pd.read_csv(output_path)
else:
    print("You don't have data from the competition yet. Please run the notebook '0_datasets_building.ipynb' first.")

Matches from the competition loaded from data/reduced/Top_5_leagues.csv


In [2]:
matches_in_Top5Leagues.head(5)

,match_id,shots_on_target_ratio_home,shots_on_target_ratio_away,average_shots_on_target_distance_home,average_shots_on_target_distance_away,shots_inside_area_ratio_home,shots_inside_area_ratio_away,pass_success_ratio_home,pass_success_ratio_away,cross_success_ratio_home,...,percentage_shots_under_pressure_home,percentage_shots_inside_area_under_pressure_home,percentage_passes_under_pressure_home,percentage_passes_inside_area_under_pressure_home,percentage_set_piece_shots_home,percentage_set_piece_shots_inside_area_home,percentage_substitutions_home,percentage_tactical_substitutions_home,percentage_tactical_changes_home,percentage_formation_changes_home
0,3890267,0.375000,0.461538,16.622853,13.820947,0.750000,0.692308,0.832013,0.755051,0.250000,...,0.250000,0.400000,0.574713,0.50,0.266667,0.300000,0.5,0.5,0.500000,0.000000
1,3890265,0.150000,0.272727,20.255117,17.607160,0.500000,0.545455,0.772727,0.828261,0.318182,...,0.769231,0.750000,0.441718,0.75,0.680000,0.727273,0.5,0.5,0.500000,0.666667
2,3890264,0.133333,0.312500,26.271612,17.542449,0.333333,0.500000,0.767241,0.704301,0.200000,...,0.500000,0.400000,0.595376,1.00,0.476190,0.500000,0.5,0.6,0.333333,0.500000
3,3890263,0.272727,0.357143,18.619963,21.381858,0.454545,0.428571,0.688000,0.620690,0.214286,...,0.444444,0.571429,0.502924,0.50,0.458333,0.454545,0.5,0.5,1.000000,1.000000
4,3890262,0.272727,0.357143,20.860604,15.628223,0.818182,0.428571,0.490909,0.657431,0.300000,...,0.222222,0.250000,0.375000,0.60,0.400000,0.600000,0.5,0.5,0.200000,0.000000


Una vez ya tenemos los datos cargados, entrenamos el mejor modelo (seleccionado tras la experimentación) y mostramos sus resultados. El mejor modelo es un Logistic Regression que utiliza una técnica PCA para el preprocesamiento de los datos (obteniendo un 64.38% de precisión en los datos de prueba); el caso es que es complicado de hacer un estudio de explicabilidad de este tipo de modelos, por lo que vamos a utilizar uno que tiene un poco menos de precisión (63.56% en los datos de prueba) pero que el estudio de explicabilidad es más entendible y en este caso es un modelo Random Forest que utiliza una técnica de oversampling en el preprocesamiento sobre los datos de entrenamiento.

In [3]:
from src.analysis import top5Leagues_best_model

best_model_Top5Leagues, evaluation_metrics, X_train_Top5Leagues, X_test_Top5Leagues, encoder_Top5Leagues, match_ids_test = top5Leagues_best_model(matches_in_Top5Leagues)
print(f"The best model for 'The top 5 leagues' is {best_model_Top5Leagues}")
print(f"The evaluation metrics for the best model are:")
evaluation_metrics.head()

The best model for 'The top 5 leagues' is RandomForestClassifier(max_depth=6, max_features='log2', n_estimators=56,
                       random_state=42)
The evaluation metrics for the best model are:


,Train Accuracy,Test Accuracy,Precision Macro,Precision Weighted,Recall Macro,Recall Weighted,F1 Macro,F1 Weighted
Random Forest Oversampling,0.859053,0.635616,0.598653,0.626413,0.60653,0.635616,0.599698,0.628597


Una vez ya tenemos el mejor modelo entrenado comenzamos el análisis de explicabilidad.